### EF Core Entity Classes

***EF Core*** lets you use any `class` to represent `data`, as long as it contains a `public property` for `each column` that you want to query.

### DbContext

After defining entity classes, the next step is to `subclass DbContext`.  
your `DbContext subclass` will contain one `DbSet<T>` property for `each entity` in your model

In [ ]:
public class NutshellContext : DbContext
{
    public DbSet<Customer> Customers { get; set; }
}

A DbContext object does three things:  
- It acts as a factory for generating `DbSet<>` objects that you can query.  

- It keeps track of any changes that you make to your entities so that you can write them back  

- It provides virtual methods that you can override to configure the connection and model.

### Configuring the connection

In [ ]:
public class NutshellContext : DbContext
{
    protected override void OnConfiguring (DbContextOptionsBuilder
                                            optionsBuilder) =>
        optionsBuilder.UseSqlServer
            (@"Server=(local);Database=Nutshell;Trusted_Connection=True");
}

`UseSqlServer` is an extension method defined in `Microsoft.EntityFramework.SqlServer`  

we can use EFCore with other database providers, including `Oracle`, `MySQL`, `PostgreSQL`, and `SQLite`.

If you’re using `ASP.NET`, you can allow its `dependency injection` framework to preconfigure `optionsBuilder`

In [ ]:
public NutshellContext (DbContextOptions<NutshellContext> options)
    : base(options) { }

### Configuring the Model

By default, ***EF Core*** is `convention based`, meaning that it `infers` the `database schema` from `your class and property names`.

In [ ]:
public class NutshellContext : DbContext
{
    protected override void OnConfiguring (DbContextOptionsBuilder
                                            optionsBuilder) =>
        optionsBuilder.UseSqlServer
            (@"Server=(local);Database=Nutshell;Trusted_Connection=True");

    public DbSet<Customer> Customers { get; set; }
    
    protected override void OnModelCreating (ModelBuilder modelBuilder) 
    {
        modelBuilder.Entity<Customer>()
            .ToTable ("Customer"); 
        /*Without this code, EF Core would map this entity to a table named 
        “Customers” rather than “Customer”, 
        because we have a DbSet<Customer> property in 
        our DbContext called Customers*/
    }
}

In [ ]:
//Fluent API

protected override void OnModelCreating (ModelBuilder modelBuilder) 
{
    modelBuilder.Entity<Customer> (entity =>
    {
        entity.ToTable ("Customer");
        entity.Property (e => e.Name)
            .HasColumnName ("Full Name") // Column name is 'Full Name'
            .IsRequired(); // Column is not nullable
    });
}

| Method    | Purpose | Example |
| -------- | ------- | ------- |
| `ToTable`  | Specify the database table<br>name for a given entity    |builder<br>.  Entity<Customer>()<br>.ToTable("Customer");
| `HasColumnName` | Specify the column name for a<br>given property     |builder<br>.Entity<Customer>()<br>.Property(c => c.Name)<br>.HasColumnName("Full Name");
|`HasKey` |Specify a key (usually that deviates<br>from convention) |builder<br>.Entity<Customer>()<br>.HasKey(c => c.CustomerNr);
|`IsRequired` |Specify that the property requires<br>a value (is not nullable) |builder<br>.Entity<Customer>()<br>.Property(c => c.Name)<br>.IsRequired();
|`HasMaxLength` |Specify the maximum length of<br>a variable-length type (usually a<br>string) whose width can vary |builder.Entity<Customer>()<br>.Property(c => c.Name)<br>.HasMaxLength(60);
|`HasColumnType` |Specify the database data type for<br>a column |builder.Entity<Purchase>()<br>.Property(p => p.Description)<br>.HasColumnType("varchar(80)");
|`Ignore` |Ignore a type |builder.Ignore<Products>();
|`Ignore` |Ignore a property of a type |builder.Entity<Customer>()<br>.Ignore(c => c.ChatName);
|`HasIndex` |Specify a property (or combination<br>of properties) should serve in the<br>database as an index |// Compound index:<br>builder.Entity<Purchase>()<br>.HasIndex(p =><br>new { p.Date, p.Price });<br>// Unique index on one property <br>builder<br>.Entity<MedicalArticle>()<br>.HasIndex(a => a.Topic)<br>.IsUnique();
|`HasOne` |See “Navigation Properties” on<br>page 438 |builder.Entity<Purchase>()<br>.HasOne(p => p.Customer)<br>.WithMany(c => c.Purchases);
|`HasMany` |See “Navigation Properties” on<br>page 438 |builder.Entity<Customer>()<br>.HasMany(c => c.Purchases)<br>.WithOne(p => p.Customer);

### Creating the database

***EF Core*** supports a `code-first` approach, which means that you can start by `defining entity classes` and then ask ***EF Core*** to `create the database`.

EFCore can create database in two ways:
- call the following method on a DbContext instance
`dbContext.Database.EnsureCreated();`  

- use ***EFCore***'s `migrations`

`Install-Package Microsoft.EntityFrameworkCore.Tools`  

`Add-Migration InitialCreate`  

`Update-Database`

In [ ]:
using (var dbContext = new NutshellContext())
{
    Console.WriteLine (dbContext.Customers.Count());
    // Executes "SELECT COUNT(*) FROM [Customer] AS [c]"

    Customer cust = new Customer()
    {
        Name = "Sara Wells"
    };
    dbContext.Customers.Add (cust);
    dbContext.SaveChanges(); // Writes changes back to database


    Customer customer = dbContext.Customers
    .Single (c => c.Name == "Sara Wells");
    cust.Name = "Dr. Sara Wells";
    dbContext.SaveChanges();
}

### Object Tracking

A ***DbContext*** instance keeps `track` of all the entities it `instantiates`  
  
it can `feed the same ones back` to you whenever you request the `same rows` in a table. (هر زمان که ردیف‌های مشابهی را در جدول درخواست می‌کنید، همان‌ها به شما باز می‌گردند.)

 در طول عمر خود هرگز دو موجودیت مجزا که به یک ردیف در یک جدول اشاره می کنند (که در آن یک ردیف با کلید اصلی مشخص می شود) منتشر نمی کند.

This capability is called `object tracking`

### Disposing DbContext

<div dir="rtl" style="width:90%; margin:auto">
کلاس DbContext از اینترفیس IDisposable پیروی می‌کند که به معنای این است که شامل متد Dispose() برای آزادسازی منابع است.
<br>

اگرچه DbContext قابلیت Disposing دارد، اما در بیشتر موارد نیازی به صراحتاً فراخوانی Dispose() نیست زیرا EF Core به صورت خودکار اتصالات به دیتابیس را بسته و منابع را آزاد می‌کند، مخصوصاً پس از پایان یافتن دریافت نتایج از یک پرس و جو.
<br>

اما، در مواردی که منابع به طور کامل توسط دیگر کامپوننت‌ها آزاد نمی‌شوند یا شما خودتان به صورت دستی عملیاتی را کنترل می‌کنید که ممکن است اتصال باز بماند، استفاده از Dispose() می‌تواند به آزادسازی منابع کمک کند و از نشت منابع جلوگیری کند.
<div>

### فرآیند کوئری و بازیابی اطلاعات

<div dir="rtl" style="width:80%; margin:auto;">
<ol>
<li>
<p>
<strong>شروع کوئری</strong>:
 EF Core ابتدا با ارسال درخواستی به دیتابیس شروع به کار می‌کند تا یک ردیف داده را بازیابی کند.
 </p>
 </li>
 <li>
 <p>
 <strong>خواندن کلید اصلی</strong>:
  پس از دریافت ردیف داده، EF Core کلید اصلی (Primary Key) آن ردیف را می‌خواند. کلید اصلی یک شناسه منحصر به فرد برای هر رکورد در جدول داده است و به EF Core کمک می‌کند تا هر شیء (Entity) را به طور یکتا شناسایی کند.
  </p>
  </li>
  <li>
  <p>
  <strong>جستجو در کش</strong>:
   EF Core سپس با استفاده از کلید اصلی خوانده شده، در کش انتیتی‌های موجود در <code>DbContext</code> جستجو می‌کند. کش انتیتی محلی است که EF Core نمونه‌هایی از داده‌ها را که پیش‌تر بازیابی شده‌اند، نگهداری می‌کند.
   </p>
   </li>
   <li>
   <p>
   <strong>بررسی و بازگشت داده</strong>:
    اگر انتیتی با کلید اصلی مشابه در کش پیدا شود، EF Core بدون به‌روزرسانی هیچ مقداری، شیء موجود را بازمی‌گرداند. حتی اگر قبل از واکشی اطلاعات ،اطلاعات این سطر تغییر کرده باشد، تغییرات را در نظر می گیرد و داده های دیتابیس را نادیده می گیرد، به همین دلیل اگر اطلاعات دیتابیس از جای دیگری تغییر کرده باشند، EF Core متوجه این تغییرات نمی شود.
    </p>
    </li>
    </ol>

</div>

To get `fresh` information from the database, you must either `instantiate a new
context` or call the `Reload` method, as follows:  

`dbContext.Entry (myCustomer).Reload();`

### Change Tracking

***EFCore*** creates a `snapshot` of the state of `entities` loaded through your DbContext subclass and `compares` the `current` state to the `original` one when `SaveChanges` is called.

In [ ]:
foreach (var e in dbContext.ChangeTracker.Entries())
{
    Console.WriteLine ($"{e.Entity.GetType().FullName} is {e.State}");

    foreach (var m in e.Members)
        Console.WriteLine (
            $" {m.Metadata.Name}: '{m.CurrentValue}' modified: {m.IsModified}");
}

<div dir="rtl" style="width:80%; margin:auto;">
<strong>مدیریت تراکنش‌ها</strong>: اگر در کد شما <code>TransactionScope</code> مشخص شده باشد، EF Core این تراکنش را به رسمیت می‌شناسد و تمام دستورات SQL را در آن تراکنش اجرا می‌کند. اگر <code>TransactionScope</code> وجود نداشته باشد، EF Core به صورت خودکار یک تراکنش جدید ایجاد می‌کند و تمام دستورات SQL را در این تراکنش جدید بسته‌بندی می‌کند. این کار به حفظ یکپارچگی داده کمک می‌کند، زیرا یا تمام دستورات با موفقیت اجرا می‌شوند و تغییرات ذخیره می‌شوند، یا در صورت وجود خطا در هر یک از دستورات، تمام تغییرات به طور خودکار بازگردانده می‌شوند (Rollback).
<div>

### Navigation Properties

In [1]:
public class Customer
{
    public int ID { get; set; }
    public string Name { get; set; }
    // Child navigation property, which must be of type ICollection<T>:
    public virtual List<Purchase> Purchases {get;set;} = new List<Purchase>();
}
public class Purchase
{
    public int ID { get; set; }
    public DateTime Date { get; set; }
    public string Description { get; set; }
    public decimal Price { get; set; }
    public int? CustomerID { get; set; } // Foreign key field
    public Customer Customer { get; set; } // Parent navigation property
}

***EF Core*** is able to `infer` from these entities that `CustomerID` is a `foreign key` to the Customer table, because the name `CustomerID` follows a popular `naming convention`.
If we were to ask ***EF Core*** to create a database from these entities, it would
create a `foreign key constraint` between `Purchase.CustomerID` and `Customer.ID`.

In [ ]:
Customer cust = dbContext.Customers.Single (c => c.ID == 1);

Purchase p1 = new Purchase { Description="Bike", Price=500 };
Purchase p2 = new Purchase { Description="Tools", Price=100 };

cust.Purchases.Add (p1);
cust.Purchases.Add (p2); 

dbContext.SaveChanges();

/*
EF Core automatically writes 1 into the CustomerID column of each
of the new purchases and writes the database-generated ID for each purchase to
Purchase.ID.
*/

### Loading navigation properties

In [ ]:
using (var dbContext = new NutshellContext())
{
    var cust = dbContext.Customers.First();
    Console.WriteLine (cust.Purchases.Count); // Always 0
}

In [ ]:
/*One solution is to use the Include extension method, which instructs EF Core to
eagerly load navigation properties*/

/****eagerly load***/
using (var dbContext = new NutshellContext())
{
    var cust = dbContext.Customers
        .Include (c => c.Purchases)
        .Where (c => c.ID == 2).First();

    Console.WriteLine (cust.Purchases.Count); // Always 0
}

In [ ]:
/*Another solution is to use a projection. This technique is particularly useful when
you need to work with only some of the entity properties, because it reduces data
transfer*/
using (var dbContext = new NutshellContext())
{
    var custInfo = dbContext.Customers
        .Where (c => c.ID == 2)
        .Select (c => new
        {
            Name = c.Name,
            Purchases = c.Purchases.Select (p => new { p.Description, p.Price })
        })
        .First();
    Console.WriteLine (custInfo.Purchases.Count); // Always 0
}

In [ ]:
/*Explicit Loading*/
using (var dbContext = new NutshellContext())
{
    var cust = dbContext.Customers
        .Where (c => c.ID == 2).First();

    dbContext.Entry(cust).Collection(a => a.Purchases).Load();
}

#### Lazy loading

<div dir="rtl" style="width:80%; margin:auto;">
داده‌های مرتبط را تنها در زمانی که به صورت فعال به آن‌ها دسترسی پیدا می‌شود، بارگذاری می‌کند. این روش به صورت خودکار و تنبل انجام می‌شود، به این معنی که داده‌های مرتبط بارگذاری نمی‌شوند مگر اینکه صریحاً به آن‌ها در کد برنامه دسترسی پیدا شود.
<br>
<h4>نحوه عملکرد Lazy Loading</h4>
<p>برای فعال کردن lazy loading در EF Core، شما نیاز به انجام دو کار دارید:</p>

<ol><li>نصب بسته NuGet <code>Microsoft.EntityFrameworkCore.Proxies</code>.</li><li>فعال‌سازی پروکسی‌ها در تنظیمات <code>DbContext</code> خود با استفاده از متد <code>UseLazyLoadingProxies()</code>.</li></ol>
<p>مدل‌های انتیتی باید دارای خصوصیات ناوبری که می‌خواهید به صورت تنبل بارگذاری شوند، به عنوان <code>virtual</code> تعریف شوند. این امر EF Core را قادر می‌سازد که پروکسی‌هایی را برای این خصوصیات ایجاد کند که می‌توانند به صورت خودکار داده‌های مرتبط را بارگذاری کنند.</p>
</div>

In [ ]:
public class Blog
{
    public int BlogId { get; set; }
    public string Url { get; set; }
    public virtual ICollection<Post> Posts { get; set; }
}

public class Post
{
    public int PostId { get; set; }
    public string Title { get; set; }
    public string Content { get; set; }
    public int BlogId { get; set; }
    public virtual Blog Blog { get; set; }
}

protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
{
    optionsBuilder.UseLazyLoadingProxies();
}



### Deferred Execution

<div dir="rtl" style="width:80%; margin:auto;">
این ویژگی اجازه می‌دهد که پرس و جوها (queries) به تدریج ساخته شوند و تا زمانی که نیاز به داده‌ها نباشد، اجرا نشوند. 
<br>
<h4>زیرپرس و جوها در EF Core</h4>
<p>زیرپرس و جوها در EF Core یک جنبه خاص دارند: زمانی که یک زیرپرس و جو در داخل عبارت <code>Select</code> ظاهر می‌شود، این زیرپرس و جو همزمان با پرس و جوی اصلی (main outer query) اجرا می‌شود. این به این دلیل است که EF Core می‌خواهد از رفت و آمدهای بیش از حد (excessive round-tripping) به پایگاه داده جلوگیری کند.</p>

<h4>تفاوت با پرس و جوهای محلی</h4>
<p>در پرس و جوهای محلی (مانند کار با لیست‌ها یا آرایه‌ها در C#)، زمانی که یک زیرپرس و جو در عبارت <code>Select</code> وجود داشته باشد، شما با اجرای به تعویق افتاده دوباره مواجه می‌شوید. این به این معناست که اگر شما دنباله نتایج خارجی (outer result sequence) را مرور کنید اما هرگز دنباله‌های داخلی (inner sequences) را مرور نکنید، زیرپرس و جو هرگز اجرا نمی‌شود.</p>
</div>

In [ ]:
using (var dbContext = new NutshellContext ())
{
    var query = from c in dbContext.Customers
                select
                    from p in c.Purchases
                    select new { c.Name, p.Price };

    foreach (var customerPurchaseResults in query)
        foreach (var namePrice in customerPurchaseResults)
            Console.WriteLine ($"{ namePrice.Name} spent { namePrice.Price}");
    /*
    پرس و جو بالا با رسیدن به اولین دستور 
    foreach
     در یک رفت و برگشت اجرا می شود
    */
}

In [ ]:
using (var dbContext = new NutshellContext ())
{
    foreach (Customer c in dbContext.Customers.ToArray())
        if (myWebService.HasBadCreditHistory (c.ID))
            foreach (Purchase p in c.Purchases) // Another SQL round trip
                Console.WriteLine (c.Name + " spent " + p.Price);
}